In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)

### liftover hg19 to hg38

In [ ]:
# UPLIFTEN
# get the variant positions
zcat output/features/annotated/hg19/SNVs.hg19.combined.txt.gz | cut -f 2-3 |  gzip -c > output/predictions/lifted/up/hg19.positions.txt.gz

# Add ReMM score to the positions; 
# ReMM is 1-based, positions also, since stem from VCF annotated files
awk 'BEGIN{ OFS="\t"} NR==FNR{ a[$1 FS $2];next} ("chr"$1 FS $2) in a {print "chr"$1, $2, $3} ' <(zcat output/predictions/lifted/up/hg19.positions.txt.gz) <(zcat input/variants/hg19/ReMM.v0.3.1.tsv.gz) |gzip -c  > output/predictions/lifted/up/hg19.positions.remm.txt.gz

# lifover requires a bed file so -1
zcat  output/predictions/lifted/up/hg19.positions.remm.txt.gz  | awk 'BEGIN{ OFS="\t"}{ print $1,$2-1,$2,$1":"$2-1":"$3 }' | gzip -c > output/predictions/lifted/up/hg19.positions.remm.bed.gz

# liftover
liftOver output/predictions/lifted/up/hg19.positions.remm.bed.gz input/variants/hg38/hg19ToHg38.over.chain.gz >( gzip -c > output/predictions/lifted/up/hg19.positions.remm.lifted.bed.gz ) >( gzip -c > output/predictions/lifted/up/failed.bed.gz)

# after liftover add 1 to start position
zcat  output/predictions/lifted/up/hg19.positions.remm.lifted.bed.gz | awk 'BEGIN{ OFS="\t"}{ print $1,$2+1,$4}' | gzip -c > output/predictions/lifted/up/hg19.positions.remm.lifted.txt.gz






In [2]:

# read in the ReMMM scores for hg19 variants (from forder up, since same as thre)
file = 'output/predictions/lifted/up/hg19.positions.remm.txt.gz'
_df19 = pd.read_csv(file,sep='\t',header = None)

# read i the lifted hg38 variants 
file = 'output/predictions/lifted/_down/hg38.predictions.lifted.txt.gz'
_df38 = pd.read_csv(file,sep='\t',header = None)

# create index column
_df38[3]=_df38[0].astype(str)+'-'+_df38[1].astype(str)
_df19[3]=_df19[0].astype(str)+'-'+_df19[1].astype(str)

# set index to join on 
_df38 = _df38.set_index(_df38[3])
_df19 = _df19.set_index(_df19[3])

# inner join the data frames on index
_f = _df38.join(_df19,lsuffix='pos',rsuffix='up',how='inner')

# create a df with two columns: ReMM scores and parSMURF predictions
_df= pd.DataFrame()
_df['38'] = _f['2pos'].str.split(':',expand = True)[2]
_df['19']= _f['2up']

# calculate correlations
_df[['19','38']].astype(float).corr()

,19,38
19,1.000000,0.704102
38,0.704102,1.000000


In [19]:
a = _df38.drop_duplicates([0,1])

In [21]:
_df38.shape

(13878373, 4)

In [20]:
a.shape

(13856298, 4)

In [24]:
_df38.duplicated(keep=False)

KeyboardInterrupt: 

In [22]:
13856298 -13878373

-22075

In [3]:
_f.shape

(12881484, 8)

### liftover  hg38 to hg19

In [ ]:
#DOWNLIFTEN
#get the variant positions + predictions
paste  -d "\t"  <(zcat output/features/annotated/hg38/SNVs.hg38.combined.txt.gz | cut -f 2-3) output/predictions/hg38/SNVs.hg38.predictions.txt | awk 'BEGIN{ OFS="\t" }{ print $1,$2,$3}' | gzip -c > output/predictions/lifted/_down/hg38.predictions.txt.gz

# lifover requires a bed file so -1
zcat  output/predictions/lifted/_down/hg38.predictions.txt.gz  | awk 'BEGIN{ OFS="\t"}{ print $1,$2-1,$2,$1":"$2-1":"$3 }' | gzip -c > output/predictions/lifted/_down/hg38.positions.bed.gz

# liftover
liftOver output/predictions/lifted/_down/hg38.positions.bed.gz input/variants/hg19/hg38ToHg19.over.chain.gz >( gzip -c > output/predictions/lifted/_down/hg38.predictions.lifted.bed.gz ) >( gzip -c > output/predictions/lifted/_down/failed.bed.gz)

# after liftover add 1 to start position
zcat  output/predictions/lifted/_down/hg38.predictions.lifted.bed.gz | awk 'BEGIN{ OFS="\t"}{ print $1,$2+1,$4}' | gzip -c > output/predictions/lifted/_down/hg38.predictions.lifted.txt.gz


In [ ]:
#  read in lifted ReMM scores
file = 'output/predictions/lifted/up/hg19.positions.remm.lifted.txt.gz'
df19 = pd.read_csv(file,sep='\t',header = None)

# read in parSMURF predictions
file = 'output/predictions/lifted/up/hg38.pred.txt.gz'
df38 = pd.read_csv(file,sep='\t',header = None)



df19[2] = df19[2].str.split(':',expand = True)[2]

df38[3]=df38[0].astype(str)+'-'+df38[1].astype(str)
df19[3]=df19[0].astype(str)+'-'+df19[1].astype(str)

df38 = df38.set_index(df38[3])
df19 = df19.set_index(df19[3])

f = df38.join(df19,lsuffix='pos',rsuffix='up',how='inner')

df= pd.DataFrame()
df['38'] = f['2pos']
df['19']= f['2up']#.str.split(':',expand = True)[2]


df[['19','38']].astype(float).corr()

### hg19 and hg38 calculated with parSMURF (w/o ReMM): liftover hg19

In [ ]:
paste  -d "\t"  <(zcat output/features/annotated/hg19/SNVs.hg19.combined.txt.gz | cut -f 2-3) output/predictions/hg19/SNVs.hg19.predictions.txt | awk 'BEGIN{ OFS="\t" }{ print $1,$2,$3}' | gzip -c > output/predictions/lifted/_up/hg19.predictions.txt.gz

zcat  output/predictions/lifted/_up/hg19.predictions.txt.gz  | awk 'BEGIN{ OFS="\t"}{ print $1,$2-1,$2,$1":"$2-1":"$3 }' | gzip -c > output/predictions/lifted/_up/hg19.predictions.bed.gz

liftOver output/predictions/lifted/_up/hg19.predictions.bed.gz input/variants/hg38/hg19ToHg38.over.chain.gz >( gzip -c > output/predictions/lifted/_up/hg19.predictions.lifted.bed.gz ) >( gzip -c > output/predictions/lifted/_up/hg19.falied.bed.gz)

zcat  output/predictions/lifted/_up/hg19.predictions.lifted.bed.gz | awk 'BEGIN{ OFS="\t"}{ print $1,$2+1,$4}' | gzip -c > output/predictions/lifted/_up/hg19.predictions.lifted.txt.gz




In [ ]:
# read in lifted parSMURF predictions for hg19
file = 'output/predictions/lifted/_up/hg19.predictions.lifted.txt.gz'
df19_0 = pd.read_csv(file,sep='\t',header = None)

# read in  parSMURF predictions for hg38

file = 'output/predictions/lifted/_up/hg38.predictions.txt.gz'
df38_0 = pd.read_csv(file,sep='\t',header = None)


df19_0[2] = df19_0[2].str.split(':',expand = True)[2]

df38_0[3]=df38_0[0].astype(str)+'-'+df38_0[1].astype(str)
df19_0[3]=df19_0[0].astype(str)+'-'+df19_0[1].astype(str)

df38_0 = df38_0.set_index(df38_0[3])
df19_0 = df19_0.set_index(df19_0[3])

f_0 = df38_0.join(df19_0,lsuffix='pos',rsuffix='up',how='inner')

df_0= pd.DataFrame()
df_0['38'] = f_0['2pos']
df_0['19']= f_0['2up']#.str.split(':',expand = True)[2]


df_0[['19','38']].astype(float).corr()

### hg19 and hg38 calculated with parSMURF (w/o ReMM): liftover hg38

In [ ]:
paste  -d "\t"  <(zcat output/features/annotated/hg38/SNVs.hg38.combined.txt.gz | cut -f 2-3) output/predictions/hg38/SNVs.hg38.predictions.txt | awk 'BEGIN{ OFS="\t" }{ print $1,$2,$3}' | gzip -c > output/predictions/lifted/_up/hg38.predictions.txt.gz

zcat  output/predictions/lifted/_up/hg38.predictions.txt.gz  | awk 'BEGIN{ OFS="\t"}{ print $1,$2-1,$2,$1":"$2-1":"$3 }' | gzip -c > output/predictions/lifted/_up/hg38.predictions.bed.gz

liftOver output/predictions/lifted/_up/hg38.predictions.bed.gz input/variants/hg19/hg38ToHg19.over.chain.gz >( gzip -c > output/predictions/lifted/_up/hg38.predictions.lifted.bed.gz ) >( gzip -c > output/predictions/lifted/_up/hg38.falied.bed.gz)

zcat  output/predictions/lifted/_up/hg38.predictions.lifted.bed.gz | awk 'BEGIN{ OFS="\t"}{ print $1,$2+1,$4}' | gzip -c > output/predictions/lifted/_up/hg38.predictions.lifted.txt.gz



In [ ]:
# read in lifted parSMURF predictions for hg38

file = 'output/predictions/lifted/_up/hg38.predictions.lifted.txt.gz'
df19_1 = pd.read_csv(file,sep='\t',header = None)

# read in parSMURF predictions for hg19

file = 'output/predictions/lifted/_up/hg19.predictions.txt.gz'
df38_1 = pd.read_csv(file,sep='\t',header = None)


df19_1[2] = df19_1[2].str.split(':',expand = True)[2]

df38_1[3]=df38_1[0].astype(str)+'-'+df38_1[1].astype(str)
df19_1[3]=df19_1[0].astype(str)+'-'+df19_1[1].astype(str)

df38_1 = df38_1.set_index(df38_1[3])
df19_1 = df19_1.set_index(df19_1[3])

f = df38_1.join(df19_1,lsuffix='pos',rsuffix='up',how='inner')

df_1= pd.DataFrame()
df_1['38'] = f_1['2pos']
df_1['19']= f_1['2up']#.str.split(':',expand = True)[2]


df_1[['19','38']].astype(float).corr()

### liftover hg38 to hg19 but bash join

In [ ]:

join -j 1 <(zcat output/predictions/lifted/down/hg38.predictions.lifted.bed.gz | sort -k1,1 -k2,2n -k3,3n | awk 'BEGIN{ OFS="\t"} {split($0,a,":"); printf("%s:%015d\t%s\t%s\t%s\n",$1,$3,$1,$2,a[3]) }' ) <(zcat input/variants/hg19/ReMM.v0.3.1.tsv.gz | tail -n +3 | awk 'BEGIN{ OFS="\t"}{ printf("chr%s:%015d\t%s\n",$1,$2,$3) }') | gzip -c > output/predictions/lifted/down/joined_kor.txt.gz


zcat  output/predictions/lifted/down/joined_kor.txt.gz | awk 'BEGIN{ OFS="\t"}{ print $5,$6}' | gzip -c > output/predictions/lifted/down/joined_korr.txt.gz


In [15]:
file = 'output/predictions/lifted/down/joined_kor.txt.gz'
df0 = pd.read_csv(file,sep=' ',header = None, usecols  =[3,4])


In [16]:
df0.head()

,3,4
0,0.035974,0.018
1,0.021006,0.178
2,0.062250,0.012
3,0.510135,0.166
4,0.000386,0.016


In [17]:
df0.corr()

,3,4
3,1.000000,0.690586
4,0.690586,1.000000
